In [20]:
import pandas as pd
import numpy as np
import requests
import csv
import string

In [21]:
df = pd.read_csv(
                "https://raw.githubusercontent.com/pirosbogar/advent_of_code_2022/main/inputs/04/input.txt", 
                 sep=',',
                 skip_blank_lines=False, header = None, names=['elf1', 'elf2'])

In [71]:
df.head()

,elf1,elf2
0,14-28,13-28
1,72-81,82-91
2,4-4,6-95
3,47-49,48-59
4,26-36,37-76


#### In how many assignment pairs does one range fully contain the other?

In [38]:
elf1_split = df["elf1"].str.split("-", n = 1, expand = True)
elf2_split = df["elf2"].str.split("-", n = 1, expand = True)

In [39]:
print(elf1_split.head())
print(elf1_split.dtypes)

    0   1
0  14  28
1  72  81
2   4   4
3  47  49
4  26  36
0    object
1    object
dtype: object


In [40]:
elf1_split.dtypes

0    object
1    object
dtype: object

In [41]:
elf1_split.columns = ['space1', 'space2']
elf2_split.columns = ['space1', 'space2']

In [42]:
elf1_split.head()

,space1,space2
0,14,28
1,72,81
2,4,4
3,47,49
4,26,36


In [46]:
elf1_split = elf1_split[['space1', 'space2']].astype(int)

elf2_split = elf2_split[['space1', 'space2']].astype(int)

In [47]:
print(elf1_split.head())
print(elf1_split.dtypes)

   space1  space2
0      14      28
1      72      81
2       4       4
3      47      49
4      26      36
space1    int32
space2    int32
dtype: object


In [53]:
elf1_result = []

for a, b in zip(elf1_split.space1, elf1_split.space2):
    elf1_result.append(np.arange(a, int(b)+1))
    

In [54]:
result[:5]

[array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]),
 array([72, 73, 74, 75, 76, 77, 78, 79, 80, 81]),
 array([4]),
 array([47, 48, 49]),
 array([26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36])]

In [55]:
elf2_result = []

for a, b in zip(elf2_split.space1, elf2_split.space2):
    elf2_result.append(np.arange(a, int(b)+1))

In [56]:
pd.DataFrame(np.concatenate(elf1_result))

,0
0,14
1,15
2,16
3,17
4,18
...,...
36625,91
36626,92
36627,93
36628,94


In [60]:
df2 = pd.DataFrame(zip(elf1_result, elf2_result), columns = ['elf1', 'elf2'])

In [61]:
df2.head()

,elf1,elf2
0,"[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2...","[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 2..."
1,"[72, 73, 74, 75, 76, 77, 78, 79, 80, 81]","[82, 83, 84, 85, 86, 87, 88, 89, 90, 91]"
2,[4],"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1..."
3,"[47, 48, 49]","[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]"
4,"[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]","[37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 4..."


In [62]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [65]:
contains = []

for a, b in zip(df2.elf1, df2.elf2):
    if intersection(a, b) == list(a):
        contains.append(1)
    elif intersection(a, b) == list(b):
        contains.append(1)
    else:
        contains.append(0)

In [66]:
contains[:10]

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [67]:
df3 = df2
df3['contains'] = contains

In [68]:
df3.head()

,elf1,elf2,contains
0,"[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2...","[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 2...",1
1,"[72, 73, 74, 75, 76, 77, 78, 79, 80, 81]","[82, 83, 84, 85, 86, 87, 88, 89, 90, 91]",0
2,[4],"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",0
3,"[47, 48, 49]","[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]",0
4,"[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]","[37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 4...",0


In [70]:
df3['contains'].sum()

305

#### In how many assignment pairs do the ranges overlap?

In [75]:
len(intersection([1,2,3], [4,5,6]))

0

In [82]:
overlap = []

for a, b in zip(df2.elf1, df2.elf2):
    if len(intersection(a, b)):
        overlap.append(1)
    else:
        overlap.append(0)

In [83]:
overlap[:5]

[1, 0, 0, 1, 0]

In [84]:
df4 = df2
df4['overlap'] = overlap
df4.head()

,elf1,elf2,contains,overlap
0,"[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2...","[13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 2...",1,1
1,"[72, 73, 74, 75, 76, 77, 78, 79, 80, 81]","[82, 83, 84, 85, 86, 87, 88, 89, 90, 91]",0,0
2,[4],"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",0,0
3,"[47, 48, 49]","[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]",0,1
4,"[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]","[37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 4...",0,0


In [85]:
df4['overlap'].sum()

811